<center style="font-size: 26px"> <b>Проверка уровня знаний Python</b></center>

In [ ]:
# обеспечиваем совместимость с Python 2 и 3
from __future__ import (absolute_import, division, print_function, unicode_literals)

# отключаем предупреждения дистрибутива Anaconda
import warnings
warnings.simplefilter('ignore')

**Описание датасета:**   
[Датасет вакансий с платформы HH.ru на Kaggle.com](https://www.kaggle.com/datasets/pavfedotov/heaadhunter-vacancies?resource=download&select=df2021-08-03.csv)

Нужный файл - df2021-08-03.csv.

- *vacancy* - наименование вакансии
- *url* - ссылка на вакансию
- *created* - дата и время создания
- *has_test* - наличие тестового задания в вакансии
- *salary_from* - нижняя граница значения заработной платы
- *salary_to* - верхняя граница значения заработной платы
- *currency* - валюта заработной платы
- *experience* - требуемый опыт
- *schedule* - тип рабочего графика
- *skills* - требуемые навыки
- *employer* - наименование работодателя
- *area* - наименование города
- *description* - описание вакансии

#### Загрузите датасет

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("df2021-08-03.csv")


In [ ]:
data.head()

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01T13:02:48+0300,False,120000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09T08:13:01+0300,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21T15:24:16+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст..."
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28T12:57:57+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31T17:10:49+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ..."


### Практические задания:

#### 1. Изучите методы для сбора данных курсов валют в [API Центрального банка Российской Федерации](https://www.cbr.ru/development/SXML/) . Используя запросы к API, приведите значения заработной платы к рублям одним из вариантов: 1) простой вариант - на текущую дату  2) вариант посложнее - на дату создания (created).

In [ ]:
data['salary_to'] = pd.to_numeric(data['salary_to'], errors='coerce')
data['salary_from'] = pd.to_numeric(data['salary_from'], errors='coerce')

In [ ]:
data['salary_to'].dtype

dtype('float64')

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

**1) На текущую дату**

In [ ]:
url = 'http://www.cbr.ru/scripts/XML_daily.asp/'
response = requests.get(url)
response.status_code

200

In [ ]:
soup = BeautifulSoup(response.text, 'xml')

eur_info = soup.find('Valute', {'ID': "R01239"}).find('Value').text

usd_info = soup.find('Valute', {'ID': "R01235"}).find('Value').text

92,3892


In [ ]:
data.loc[data['currency'] == 'EUR', 'salary_to'] *= float(eur_info.replace(',', '.'))
data.loc[data['currency'] == 'USD', 'salary_to'] *= float(usd_info.replace(',', '.'))

In [ ]:
data.loc[data['currency'] == 'EUR', 'salary_from'] *= float(eur_info.replace(',', '.'))
data.loc[data['currency'] == 'USD', 'salary_from'] *= float(usd_info.replace(',', '.'))

**2) На дату создания**

Как мы видим сервер нам возвращает ОТВЕТ 404 , который говорит , что нет такой страницы. Но если мы выполним запрос через браузер , то все нормально.

Такая ошибка происходит из-за того , что сервер думает , что мы робот , так по умолчанию User-Agent отправляемого запроса равен python-requests/2.22.0. Чтобы сервер нам возвращал нормальный ответ , нужно указать правильный User-Agent, который эмулирует действия пользователя.



Теперь разберемся для чего нужен _xsrf . Когда мы логинимся на сайте hh.ru , то мы просто вводим логин и пароль и все. Но при этом при POST запросе на сервер кроме введенных нами значений передается и параметр _xsrf, который нужен для защиты от CSRF-атак. Так вот значение этого параметра сервер hh.ru генерирует автоматически и хранит его в кукис файлах в браузере и чтобы получить это значение , нам нужно использовать объект Session библиотеки requests.

Объект Session позволяет сохранять определенные параметры в запросах.Он также сохраняет файлы COOKIE во всех запросах.

In [ ]:
def get_data_from_hh(url):
  user_agent_val = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
  session = requests.Session()
  r = session.get(data_url, headers = {
    'User-Agent': user_agent_val
  })

  # Указываем referer. Иногда , если не указать , то приводит к ошибкам.
  session.headers.update({'Referer':url})

  #Хотя , мы ранее указывали наш user-agent и запрос удачно прошел и вернул
  # нам нужный ответ, но user-agent изменился на тот , который был
  # по умолчанию. И поэтому мы обновляем его.
  session.headers.update({'User-Agent':user_agent_val})

  # Получаем значение _xsrf из cookies
  _xsrf = session.cookies.get('_xsrf', domain=".hh.ru")

  post_request = session.post(url, {
      'backUrl': url,
      'username': '',
      'password': '',
      '_xsrf':_xsrf,
      'remember':'yes',
  })
  r2 = session.get(url, headers = {
  'User-Agent': user_agent_val
  })
  soup_g = BeautifulSoup(r2.text, 'lxml')
  if soup_g is not None:
    url_vacancy = soup_g.find('div', class_='main-content main-content_broad-spacing')
    if url_vacancy is not None:
      url_vacancy_f = url_vacancy.find('div', class_='bloko-columns-wrapper')
      if url_vacancy_f is not None:
        url_vacancy_f_f = url_vacancy_f.find('a', class_='bloko-link bloko-link_disable-visited')
        if url_vacancy_f_f is not None:
          res_url = url_vacancy_f_f['href']
          if res_url is not None:
            vacancy_response = requests.get('https://spb.hh.ru' + res_url, headers = {
            'User-Agent': user_agent_val
            })
            soup_v = BeautifulSoup(vacancy_response.text, 'lxml')
            desc = soup_v.find('meta', {'name': 'description'}).get('content')
            date_f = desc.find('Дата публикации')
            date_st = date_f + len('Дата публикации: ')
            date_end = date_st + 10
            date = desc[date_st:date_end]
            return date

In [ ]:
index = data.loc[(data['currency'] == 'EUR') | (data['currency'] == 'USD'), 'url'].index.values.astype(int)

In [ ]:
for data_url, iterations in zip(data.loc[(data['currency'] == 'EUR') | (data['currency'] == 'USD'), 'url'], index):
  date = get_data_from_hh(data_url)
  if date is not None:
    url_date = 'http://www.cbr.ru/scripts/XML_daily.asp?date_req=' + date
    url_text = requests.get(url_date)
    if url_text.status_code == 200:
      soup = BeautifulSoup(url_text.text, 'xml')
      if data.loc[iterations, 'currency'] == 'EUR':
        euro_info = soup.find('Valute', {'ID': "R01239"})
        if euro_info is not None:
          euro = euro_info.find('Value').text
          data.loc[iterations, 'salary_to'] *= float(euro.replace(',', '.'))
          data.loc[iterations, 'salary_from'] *= float(euro.replace(',', '.'))
      if data.loc[iterations, 'currency'] == 'USD':
        usdd_info = soup.find('Valute', {'ID': "R01235"})
        if usdd_info is not None:
          usdd = usdd_info.find('Value').text
          data.loc[iterations, 'salary_to'] *= float(usdd.replace(',', '.'))
          data.loc[iterations, 'salary_from'] *= float(usdd.replace(',', '.'))


19
https://hh.ru/applicant/vacancy_response?vacancyId=46066105
06.08.2021
EURO 86.6529
123
https://hh.ru/applicant/vacancy_response?vacancyId=46670558
30.07.2021
EURO 86.8624
142
https://hh.ru/applicant/vacancy_response?vacancyId=45657847
16.09.2021
72,8520
145
https://hh.ru/applicant/vacancy_response?vacancyId=46670557
30.07.2021
EURO 86.8624
175
https://hh.ru/applicant/vacancy_response?vacancyId=46497746
21.08.2021
74,3640
191
https://hh.ru/applicant/vacancy_response?vacancyId=45560000
02.08.2021
73,1388
371
https://hh.ru/applicant/vacancy_response?vacancyId=45795577
386
https://hh.ru/applicant/vacancy_response?vacancyId=46439735
18.08.2021
73,4753
427
https://hh.ru/applicant/vacancy_response?vacancyId=46351042
18.08.2021
EURO 86.4804
435
https://hh.ru/applicant/vacancy_response?vacancyId=46496928
21.08.2021
74,3640
437
https://hh.ru/applicant/vacancy_response?vacancyId=45468885
27.09.2021
EURO 85.6823
465
https://hh.ru/applicant/vacancy_response?vacancyId=46563619
23.08.2021
74,3640

**Работа с пропусками (количественные признаки)**

Имеется большой процент пропусков в колонках с количественными признаками 'salary_to', 'salary_from'

Используем медиану, т.к она более устойчива к выбросам

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   vacancy      1700 non-null   object 
 1   url          1700 non-null   object 
 2   created      1700 non-null   object 
 3   has_test     1700 non-null   bool   
 4   salary_from  538 non-null    float64
 5   salary_to    433 non-null    float64
 6   currency     613 non-null    object 
 7   experience   1700 non-null   object 
 8   schedule     1700 non-null   object 
 9   skills       1490 non-null   object 
 10  employer     1700 non-null   object 
 11  area         1700 non-null   object 
 12  description  1700 non-null   object 
dtypes: bool(1), float64(2), object(10)
memory usage: 161.2+ KB


In [ ]:
data['salary_from'] = data['salary_from'].fillna(data['salary_from'].median())
data['salary_to'] = data['salary_to'].fillna(data['salary_to'].median())

#### 2. Найдите вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [ ]:
req_data = data[
    (data['experience'] == 'От 3 до 6 лет') & (data['has_test'] == False)
].sort_values(by='salary_from', ascending=False).head(10)
max_salary = req_data['salary_from'].iloc[0]
max_vacancy = req_data[req_data['salary_from'] == max_salary].iloc[0]['vacancy']
max_vacancy

'Senior Python Developer (Кипр, Лимассол)'

#### 3. Сделайте рейтинг (топ-30) навыков по всем вакансиям.

In [ ]:
skills_notna = data['skills'].notna()
skills = []
for skill in data.loc[skills_notna, 'skills']:
  ext = skill.split(';')
  skills.extend(ext)
skills_series = pd.Series(data=skills).replace('', None).dropna()
top_skills = skills_series.value_counts()[:30]
for i, skill in enumerate(top_skills, start=1):
  print(f'Топ 30 навыков: \n{i, skill}')

Топ 30 навыков: 
Python              1442
Git                  586
SQL                  501
PostgreSQL           499
Django Framework     467
Linux                463
Flask                177
Docker               172
JavaScript           149
Английский язык      141
MySQL                128
ООП                  120
MongoDB              113
Redis                107
REST                  70
RabbitMQ              68
Django                62
CSS                   56
HTML                  56
Atlassian Jira        52
C++                   50
API                   49
Java                  43
MS SQL                43
Базы данных           41
Работа в команде      39
AWS                   36
Pytest                35
Celery                35
aiohttp               33
dtype: int64


#### 4. Cоставьте топ-10 городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [ ]:
data_qa = data[
    data['vacancy'].str.contains('QA') | data['vacancy'].str.contains('тест') | data['vacancy'].str.contains('Тест')]
top_area = data_qa.groupby(data_qa['area']).agg({'salary_from': 'mean'})
top = top_area.sort_values(by='salary_from', ascending=False).head(10)
print(f"Топ-10 городов: \n{top}")

Топ-10 городов: 
                   salary_from
area                          
Тамбов           150000.000000
Санкт-Петербург  122666.666667
Москва           119459.603333
Брянск           110000.000000
Димитровград     110000.000000
Краснодар        110000.000000
Нижний Новгород  110000.000000
Новосибирск      110000.000000
Ростов-на-Дону   110000.000000
Самара           110000.000000


#### 5. Напишите функцию для поиска вакансий по переданным в нее значениям заработной платы, наличия тестового задания, наличия заданного текста в наименовании вакансии, наличия заданного навыка. В результате выведите количество найденных вакансий, среднюю заработную плату по найденным вакансиям и список ссылок найденных вакансий.

In [ ]:
def search(salary_from, salary_to, has_test, text_vac, skill):
  def_data = data[
      (data['has_test'] == has_test) &
      (data['vacancy'].str.contains(text_vac)) &
      (data['skills'].str.contains(skill)) &
      (data['salary_to'] == salary_to) &
      (data['salary_from'] == salary_from)
  ]
  def_data_mean = def_data['salary_to'].mean()
  print(f"Количество найденных вакансий: {len(def_data['vacancy'])}")
  print(f"Средняя зарплата по найденным вакансиям: {def_data_mean}")
  val = [i for i in range(len(def_data['vacancy']))]
  list_of_vac = [url for url in def_data['url']]
  print("Список ссылок найденных вакансий:")
  for i, url in enumerate(list_of_vac, start=1):
    print(i, url)


#Example
search(150000.0, 220000.0, False, 'разработчик', 'Python')

Количество найденных вакансий: 12
Средняя зарплата по найденным вакансиям: 220000.0
Список ссылок найденных вакансий:
1 https://hh.ru/applicant/vacancy_response?vacancyId=46106059
2 https://hh.ru/applicant/vacancy_response?vacancyId=46582876
3 https://hh.ru/applicant/vacancy_response?vacancyId=45654903
4 https://hh.ru/applicant/vacancy_response?vacancyId=46106105
5 https://hh.ru/applicant/vacancy_response?vacancyId=40278614
6 https://hh.ru/applicant/vacancy_response?vacancyId=45244551
7 https://hh.ru/applicant/vacancy_response?vacancyId=46017566
8 https://hh.ru/applicant/vacancy_response?vacancyId=46531100
9 https://hh.ru/applicant/vacancy_response?vacancyId=43925214
10 https://hh.ru/applicant/vacancy_response?vacancyId=46106065
11 https://hh.ru/applicant/vacancy_response?vacancyId=46106111
12 https://hh.ru/applicant/vacancy_response?vacancyId=46596485


#### *Необязательно*. Усложненный вариант - сделать атрибуты функции опциональными, чтобы иметь возможность фильтрации вакансий по любой комбинации условий.